In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12789068638666036510, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14372884758905491494
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10830479416676333246
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14640891840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4902494940875033115
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout
from keras.layers.embeddings import Embedding
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model

from sklearn.model_selection import StratifiedKFold

# 랜덤 시드값 지정
np.random.seed(10)

# 데이터 불러오기 + 전처리
df_train = pd.read_csv('/content/gdrive/My Drive/보아즈미니플젝/train.csv')
df_test = pd.read_csv('/content/gdrive/My Drive/보아즈미니플젝/test.csv')

X_train, y_train = df_train.iloc[:,2:], df_train.target
X_test = df_test.iloc[:,1:]


cols_use = [c for c in X_train.columns if (not c.startswith('ps_calc_'))]

X_train = X_train[cols_use]
X_test = X_test[cols_use]

col_vals_dict = {c: list(X_train[c].unique()) for c in X_train.columns if c.endswith('_cat')}

# 임베딩 차원 확인(value count)
embed_cols = []
for c in col_vals_dict:
    if len(col_vals_dict[c])>2:
        embed_cols.append(c)
        print(c + ': %d values' % len(col_vals_dict[c])) 

ps_ind_02_cat: 5 values
ps_ind_04_cat: 3 values
ps_ind_05_cat: 8 values
ps_car_01_cat: 13 values
ps_car_02_cat: 3 values
ps_car_03_cat: 3 values
ps_car_04_cat: 10 values
ps_car_05_cat: 3 values
ps_car_06_cat: 18 values
ps_car_07_cat: 3 values
ps_car_09_cat: 6 values
ps_car_10_cat: 3 values
ps_car_11_cat: 104 values




In [ ]:
# 범주형 변수들에 대해 엔티티 임베딩
def build_embedding_network():
    
    inputs = []
    embeddings = []

    input_ps_ind_02_cat = Input(shape=(1,))
    embedding = Embedding(5, 3, input_length=1)(input_ps_ind_02_cat)
    embedding = Reshape(target_shape=(3,))(embedding)
    inputs.append(input_ps_ind_02_cat)
    embeddings.append(embedding)
    
    input_ps_ind_04_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_ind_04_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_ind_04_cat)
    embeddings.append(embedding)
    
    input_ps_ind_05_cat = Input(shape=(1,))
    embedding = Embedding(8, 5, input_length=1)(input_ps_ind_05_cat)
    embedding = Reshape(target_shape=(5,))(embedding)
    inputs.append(input_ps_ind_05_cat)
    embeddings.append(embedding)
    
    input_ps_car_01_cat = Input(shape=(1,))
    embedding = Embedding(13, 7, input_length=1)(input_ps_car_01_cat)
    embedding = Reshape(target_shape=(7,))(embedding)
    inputs.append(input_ps_car_01_cat)
    embeddings.append(embedding)
    
    input_ps_car_02_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_car_02_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_car_02_cat)
    embeddings.append(embedding)
    
    input_ps_car_03_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_car_03_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_car_03_cat)
    embeddings.append(embedding)
    
    input_ps_car_04_cat = Input(shape=(1,))
    embedding = Embedding(10, 5, input_length=1)(input_ps_car_04_cat)
    embedding = Reshape(target_shape=(5,))(embedding)
    inputs.append(input_ps_car_04_cat)
    embeddings.append(embedding)
    
    input_ps_car_05_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_car_05_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_car_05_cat)
    embeddings.append(embedding)
    
    input_ps_car_06_cat = Input(shape=(1,))
    embedding = Embedding(18, 8, input_length=1)(input_ps_car_06_cat)
    embedding = Reshape(target_shape=(8,))(embedding)
    inputs.append(input_ps_car_06_cat)
    embeddings.append(embedding)
    
    input_ps_car_07_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_car_07_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_car_07_cat)
    embeddings.append(embedding)
    
    input_ps_car_09_cat = Input(shape=(1,))
    embedding = Embedding(6, 3, input_length=1)(input_ps_car_09_cat)
    embedding = Reshape(target_shape=(3,))(embedding)
    inputs.append(input_ps_car_09_cat)
    embeddings.append(embedding)
    
    input_ps_car_10_cat = Input(shape=(1,))
    embedding = Embedding(3, 2, input_length=1)(input_ps_car_10_cat)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_ps_car_10_cat)
    embeddings.append(embedding)
    
    input_ps_car_11_cat = Input(shape=(1,))
    embedding = Embedding(104, 10, input_length=1)(input_ps_car_11_cat)
    embedding = Reshape(target_shape=(10,))(embedding)
    inputs.append(input_ps_car_11_cat)
    embeddings.append(embedding)
    
    input_numeric = Input(shape=(24,))
    embedding_numeric = Dense(16)(input_numeric) 
    inputs.append(input_numeric)
    embeddings.append(embedding_numeric)

    x = Concatenate()(embeddings)
    x = Dense(80, activation='relu')(x)
    x = Dropout(.35)(x)
    x = Dense(20, activation='relu')(x)
    x = Dropout(.15)(x)
    x = Dense(10, activation='relu')(x)
    x = Dropout(.15)(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, output)

    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model


# 리스트형식으로 변환(네트워크 구조에 맞도록!)
def preproc(X_train, X_val, X_test):

    input_list_train = []
    input_list_val = []
    input_list_test = []
    
    # 임베딩 변수들 : 범위 rescaling [0, # values)
    for c in embed_cols:
        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        input_list_train.append(X_train[c].map(val_map).values)
        input_list_val.append(X_val[c].map(val_map).fillna(0).values)
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
     
    # 나머지 변수들!
    other_cols = [c for c in X_train.columns if (not c in embed_cols)]
    input_list_train.append(X_train[other_cols].values)
    input_list_val.append(X_val[other_cols].values)
    input_list_test.append(X_test[other_cols].values)
    
    return input_list_train, input_list_val, input_list_test    

# 지니계수 정의
def ginic(actual, pred):
    n = len(actual)
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_c[-1] - (n + 1) / 2.0
    return giniSum / n
 
# 정규화된 지니계수
def gini_normalizedc(a, p):
    return ginic(a, p) / ginic(a, a)


In [ ]:
# 학습!
K = 8 # kfold
runs_per_fold = 3
n_epochs = 15

cv_ginis = []
full_val_preds = np.zeros(np.shape(X_train)[0])
y_preds = np.zeros((np.shape(X_test)[0],K))

kfold = StratifiedKFold(n_splits = K, 
                            random_state = 231, 
                            shuffle = True)    

for i, (f_ind, outf_ind) in enumerate(kfold.split(X_train, y_train)):

    X_train_f, X_val_f = X_train.loc[f_ind].copy(), X_train.loc[outf_ind].copy()
    y_train_f, y_val_f = y_train[f_ind], y_train[outf_ind]
    
    X_test_f = X_test.copy()
    
    # upsampling 적용
    pos = (pd.Series(y_train_f == 1))
    
    # positive examples 추가
    X_train_f = pd.concat([X_train_f, X_train_f.loc[pos]], axis=0)
    y_train_f = pd.concat([y_train_f, y_train_f.loc[pos]], axis=0)
    
    # 데이터 Shuffle
    idx = np.arange(len(X_train_f))
    np.random.shuffle(idx)
    X_train_f = X_train_f.iloc[idx]
    y_train_f = y_train_f.iloc[idx]
    
    # 전처리
    proc_X_train_f, proc_X_val_f, proc_X_test_f = preproc(X_train_f, X_val_f, X_test_f)
    
    # cv score에 대한 oof prediction
    val_preds = 0
    
    for j in range(runs_per_fold):
    
        NN = build_embedding_network()
        with tf.device('/gpu:0'):
          NN.fit(proc_X_train_f, y_train_f.values, epochs=n_epochs, batch_size=4096, verbose=0)
          val_preds += NN.predict(proc_X_val_f)[:,0] / runs_per_fold
          y_preds[:,i] += NN.predict(proc_X_test_f)[:,0] / runs_per_fold

        
    full_val_preds[outf_ind] += val_preds
        
    cv_gini = gini_normalizedc(y_val_f.values, val_preds)
    cv_ginis.append(cv_gini)
    print ('\nFold %i prediction cv gini: %.5f\n' %(i,cv_gini))
    
print('Mean out of fold gini: %.5f' % np.mean(cv_ginis))
print('Full validation gini: %.5f' % gini_normalizedc(y_train.values, full_val_preds))

y_pred_final = np.mean(y_preds, axis=1)

df_sub = pd.DataFrame({'id' : df_test.id, 
                       'target' : y_pred_final},
                       columns = ['id','target'])



Fold 0 prediction cv gini: 0.28916


Fold 1 prediction cv gini: 0.28105


Fold 2 prediction cv gini: 0.25646


Fold 3 prediction cv gini: 0.24545


Fold 4 prediction cv gini: 0.27679


Fold 5 prediction cv gini: 0.24595


Fold 6 prediction cv gini: 0.27546


Fold 7 prediction cv gini: 0.28167


Fold 8 prediction cv gini: 0.27337


Fold 9 prediction cv gini: 0.27674

Mean out of fold gini: 0.27021
Full validation gini: 0.26966


In [ ]:
df_sub.to_csv('/content/gdrive/My Drive/보아즈미니플젝/NN_k10.csv', index=False)
pd.DataFrame(full_val_preds).to_csv('/content/gdrive/My Drive/보아즈미니플젝/fullNN_k10.csv',index=False)